# PyironFlow demo

Uncomment the following line if you have edited the .js files

In [1]:
# !npx esbuild js/widget.jsx --minify --format=esm --bundle --outdir=static

In [2]:
%config IPCompleter.evaluation='unsafe'

import sys
from pathlib import Path
sys.path.insert(0, str(Path(Path.cwd()).parent) + '/pyiron_nodes')

## Elastic constants

In [3]:
from pyiron_workflow import Workflow   

import pyiron_nodes as pn

wf = Workflow('compute_elastic_constants')
wf.engine = pn.atomistic.engine.ase.M3GNet()
wf.bulk = pn.atomistic.structure.build.Bulk('Pb', cubic=True)
wf.elastic = pn.atomistic.property.elastic.ElasticConstants(structure=wf.bulk, engine=wf.engine) #, parameters=parameters)

# wf.elastic.pull()

In [4]:
from python.pyironflow import PyironFlow

pf = PyironFlow([wf])
pf.gui

### Some extra features

#### Get the nodes from the gui

In [5]:
import json
json.loads(pf.wf_widgets[0].gui.nodes);

#### Show the workflow widget from tab_0

In [6]:
pf.wf_widgets[0].gui

ReactFlowWidget(edges='[{"source": "engine", "sourceHandle": "engine", "target": "elastic", "targetHandle": "e…

#### Get the current workflow from the gui and visualize it (test completeness of switching between graphical and programmatic representation)

In [7]:
wf = pf.get_workflow()

pf = PyironFlow([wf])
pf.gui

## Phonons

In [8]:
from pyiron_workflow import Workflow   

import pyiron_nodes as pn

wf = Workflow('phonons')
wf.engine = pn.atomistic.engine.ase.M3GNet()
wf.bulk = pn.atomistic.structure.build.CubicBulkCell('Pb', cell_size=3)
wf.phonopy = pn.atomistic.property.phonons.CreatePhonopy(structure=wf.bulk, engine=wf.engine) #, parameters=parameters)
wf.dos = pn.atomistic.property.phonons.GetTotalDos(phonopy=wf.phonopy.outputs.phonopy)
# wf.plot = pn.

# wf.run()


In [9]:
pf = PyironFlow([wf])
pf.gui

## Built Lammps workflow from scratch

In [10]:
from pyiron_workflow import Workflow   
import pyiron_nodes as pn


In [38]:
wf = Workflow('Lammps')
wf.structure = pn.atomistic.structure.build.Bulk('Al', cubic=True)
wf.repeat = pn.atomistic.structure.transform.Repeat(structure=wf.structure, repeat_scalar=3)

wf.calculator = pn.lammps.CalcMD() # temperature=300, n_ionic_steps=10_000)
wf.potential = pn.atomistic.engine.lammps.Potential(
    structure=wf.structure, name='1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1'
)

wf.init_lammps = pn.atomistic.engine.lammps.InitLammps(
        structure=wf.repeat,
        potential=wf.potential,
        calculator=wf.calculator,
        working_directory="test2",
    )

wf.shell = pn.atomistic.engine.lammps.Shell(
        # command=ExecutablePathResolver(module="lammps", code="lammps").path(),
        working_directory=wf.init_lammps,
    )

wf.ParseLogFile = pn.atomistic.engine.lammps.ParseLogFile(
    log_file=wf.shell.outputs.log
)
wf.ParseDumpFile = pn.atomistic.engine.lammps.ParseDumpFile(
    dump_file=wf.shell.outputs.dump
)
wf.Collect = pn.atomistic.engine.lammps.Collect(
    out_dump=wf.ParseDumpFile.outputs.dump,
    out_log=wf.ParseLogFile.outputs.log,
    calc_mode='md',
)

wf.get_energy_pot = pn.atomistic.engine.lammps.GetEnergyPot(generic=wf.Collect)

out = wf.run()

In [12]:
wf.get_energy_pot.pull()[0:-1]

array([-362.87999968, -358.5129855 , -358.4579939 , -358.31296039,
       -358.55784106, -359.02478623, -358.31887295, -358.83137511,
       -358.65856012])

In [13]:
from python.pyironflow import PyironFlow

pf = PyironFlow([wf])
pf.gui

### Use Lammps Macro

In [39]:
wf = Workflow('lammps_macro')
wf.bulk = pn.atomistic.structure.build.CubicBulkCell('Pb', cell_size=3)
wf.lammps = pn.atomistic.engine.lammps.Code(structure=wf.bulk)

wf.bulk.pull()
wf.lammps.Collect.pull() # works
wf.run() # does not work when previous line is commented out

Lammps:  structure (UserInput):
Inputs ['user_input']
OutputsWithInjection ['user_input']
InputSignals ['run', 'accumulate_and_run']
OutputSignals ['ran']


{'lammps__generic': OutputCalcMD(energies_pot=array([-218.981703]), energies_kin=array([0.]), forces=array([[[-3.33066907e-16, -2.87964097e-16, -3.81639165e-16],
         [-4.99600361e-16, -2.79290480e-16, -5.53376789e-16],
         [-3.98986399e-16, -5.68989300e-16, -4.64905892e-16],
         [-3.08780779e-16, -3.08780779e-16, -6.52256027e-16],
         [-4.30211422e-16, -4.51028104e-16, -3.81329587e-16],
         [-2.91433544e-16, -3.07046055e-16,  7.45931095e-17],
         [-2.53269627e-16, -3.88578059e-16,  1.75207071e-16],
         [-4.02455846e-16, -4.09394740e-16, -2.75052213e-16],
         [-1.66533454e-16, -1.68268177e-16,  4.80043602e-16],
         [-1.94289029e-16, -2.44596010e-16,  7.97972799e-16],
         [-2.70616862e-16, -1.59594560e-16,  6.17561557e-16],
         [-3.71230824e-16, -3.46944695e-16,  5.71248297e-16],
         [-3.46944695e-16, -1.91268946e-16, -4.05925293e-16],
         [-3.12250226e-16, -1.78676518e-16, -4.37150316e-16],
         [-5.34294831e-16,  4.23

In [40]:
pf = PyironFlow([wf])
pf.gui